In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle as pk
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

dataseta = pd.read_csv('Train.tsv',sep='\t',dtype=object)
dataset_test1 = pd.read_csv('Test.tsv',sep='\t',dtype=object)
dataseta.drop(dataseta[dataseta['Payer']=='*'].index,inplace=True)
dataseta.drop(columns='PercentofPayAmount',inplace=True)
#dataseta.info()
#dataseta.describe()
Convert_dict = {'Payer': str, 
                'Debit_CreditInd.': str,
                'DaystoPay': float
               } 
  
dataseta = dataseta.astype(Convert_dict) 
print(dataseta.dtypes) 
dataseta.head(5)
dataseti=dataseta.groupby(['Payer','Debit_CreditInd.'])['DaystoPay'].mean()
datasetii = pd.DataFrame(dataseti).reset_index()
#datasetii.head(20)
inner_join = pd.merge(dataseta,  
                      datasetii,  
                      on =['Payer','Debit_CreditInd.'],  
                      how ='inner') 
inner_join['rank'] = inner_join.groupby(['Payer','Debit_CreditInd.'])['DaystoPay_x'].rank()
dataset_ready=inner_join[['Payer','Debit_CreditInd.','rank','DaystoPay_x']]
dataset=dataset_ready
X1=dataset[['Payer','Debit_CreditInd.','rank']]
dataset_test=dataset_test1
X1['Debit_CreditInd.']=dataset['Debit_CreditInd.'].apply(lambda x: 1 if x == 'S' else 0)
dataset_test['Debit_CreditInd.']=dataset_test['Debit_CreditInd.'].apply(lambda x: 1 if x == 'S' else 0)
y = dataset.iloc[:, -1].values
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
file = open("models.pkl", "wb")
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse=False,handle_unknown='ignore'), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X1))
pk.dump(ct, file)
from sklearn.ensemble import RandomForestClassifier
classifier1 = RandomForestClassifier(n_estimators = 75, criterion = 'entropy', random_state = 0,bootstrap=True,
                                    ccp_alpha=0.0, class_weight=None,
                        max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, 
                       n_jobs=None, oob_score=False, verbose=0,
                       warm_start=False)
classifier1.fit(X, y)
pk.dump(classifier1, file)
file.close()

dataset_test['rank']=1

file = open("models.pkl", "rb")
trained_encoder = pk.load(file)  #Pickle file first load the OneHotEncoder 
trained_model_for_prediction = pk.load(file)
X_test_encoded=np.array(trained_encoder.transform(dataset_test))
y_predict=trained_model_for_prediction.predict(X_test_encoded)
dataset_test['DaystoPay']=y_predict
print(dataset_test)
file.close()
y_pred=classifier1.predict(X)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import average_precision_score,classification_report
from sklearn.metrics import f1_score
#average_precision = average_precision_score(y_test_in, y_pred)
cm = confusion_matrix(y,y_pred)
print( accuracy_score(y,y_pred))
print(classification_report(y,y_pred))

Payer                object
Debit_CreditInd.     object
DaystoPay           float64
dtype: object


C:\Users\deesaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


      Payer  Debit_CreditInd.  rank  DaystoPay
0    100030                 0     1        7.0
1    100030                 1     1       39.0
2  70000010                 1     1        0.0
3    300767                 1     1        0.0
4    100040                 0     1       22.0
5    100059                 0     1       11.0
6    301096                 1     1        4.0
7    301097                 1     1       16.0
8    301136                 1     1        2.0
9  70000000                 1     1        0.0
0.9999887459626141
              precision    recall  f1-score   support

    -43902.0       1.00      1.00      1.00         1
    -43899.0       1.00      1.00      1.00         1
    -43898.0       1.00      1.00      1.00         1
    -43896.0       1.00      1.00      1.00         1
    -43893.0       1.00      1.00      1.00         1
    -43887.0       1.00      1.00      1.00         1
    -43886.0       1.00      1.00      1.00         1
    -43885.0       1.00      1.

In [10]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle as pk
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import xgboost as xgb

dataseta = pd.read_csv('Train.tsv',sep='\t',dtype=object)
dataset_test1 = pd.read_csv('Test.tsv',sep='\t',dtype=object)
dataseta.drop(dataseta[dataseta['Payer']=='*'].index,inplace=True)
dataseta.drop(columns='DaystoPay',inplace=True)
#dataseta.info()
#dataseta.describe()
Convert_dict = {'Payer': str, 
                'Debit_CreditInd.': str,
                'PercentofPayAmount': float
               } 
  
dataseta = dataseta.astype(Convert_dict) 
#print(dataseta.dtypes) 
#dataseta.head(5)
dataseti=dataseta.groupby(['Payer','Debit_CreditInd.'])['PercentofPayAmount'].mean()
datasetii = pd.DataFrame(dataseti).reset_index()
#datasetii.head(20)

inner_join = pd.merge(dataseta,  
                      datasetii,  
                      on =['Payer','Debit_CreditInd.'],  
                      how ='inner') 


inner_join['rank'] = inner_join.groupby(['Payer','Debit_CreditInd.'])['PercentofPayAmount_x'].rank()
dataset_ready=inner_join[['Payer','Debit_CreditInd.','rank','PercentofPayAmount_x']]

dataset_ready.head()

dataset=dataset_ready
X1=dataset[['Payer','Debit_CreditInd.','rank']]
dataset_test=dataset_test1
X1['Debit_CreditInd.']=dataset['Debit_CreditInd.'].apply(lambda x: 1 if x == 'S' else 0)
dataset_test['Debit_CreditInd.']=dataset_test['Debit_CreditInd.'].apply(lambda x: 1 if x == 'S' else 0)
y = dataset.iloc[:, -1].values
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
file = open("models.pkl", "wb")
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse=False,handle_unknown='ignore'), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X1))
pk.dump(ct, file)


C:\Users\deesaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ValueError: Unknown label type: 'continuous'

In [65]:
print(dataseta.shape)
print(inner_join.shape)
print(X1)




(88857, 3)
(88857, 5)
          Payer  Debit_CreditInd.  rank
0      10000100                 1   1.0
1      10000100                 1   2.0
2        100030                 0   1.0
3        100030                 1  18.0
4        100030                 1  16.5
...         ...               ...   ...
88852  70000010                 1  47.0
88853  70000010                 1  47.0
88854  70000010                 1  47.0
88855  70000010                 1  47.0
88856  70000010                 1  47.0

[88857 rows x 3 columns]


,Payer,Debit_CreditInd.
0,100030,0
1,100030,1
2,70000010,1
3,300767,1
4,100040,0
5,100059,0
6,301096,1
7,301097,1
8,301136,1
9,70000000,1


In [69]:

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle as pk
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import xgboost as xgb

dataseta = pd.read_csv('Train.tsv',sep='\t',dtype=object)
dataset_test1 = pd.read_csv('Test.tsv',sep='\t',dtype=object)
dataseta.drop(dataseta[dataseta['Payer']=='*'].index,inplace=True)
dataseta.drop(columns='PercentofPayAmount',inplace=True)
#dataseta.info()
#dataseta.describe()
Convert_dict = {'Payer': str, 
                'Debit_CreditInd.': str,
                'DaystoPay': float
               } 
  
dataseta = dataseta.astype(Convert_dict) 
print(dataseta.dtypes) 
dataseta.head(5)
dataseti=dataseta.groupby(['Payer','Debit_CreditInd.'])['DaystoPay'].mean()
datasetii = pd.DataFrame(dataseti).reset_index()
#datasetii.head(20)
inner_join = pd.merge(dataseta,  
                      datasetii,  
                      on =['Payer','Debit_CreditInd.'],  
                      how ='inner') 
inner_join['rank'] = inner_join.groupby(['Payer','Debit_CreditInd.'])['DaystoPay_x'].rank()
dataset_ready=inner_join[['Payer','Debit_CreditInd.','rank','DaystoPay_x']]
dataset=dataset_ready
X1=dataset[['Payer','Debit_CreditInd.','rank']]
dataset_test=dataset_test1
X1['Debit_CreditInd.']=dataset['Debit_CreditInd.'].apply(lambda x: 1 if x == 'S' else 0)
dataset_test['Debit_CreditInd.']=dataset_test['Debit_CreditInd.'].apply(lambda x: 1 if x == 'S' else 0)
y = dataset.iloc[:, -1].values
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
file = open("models.pkl", "wb")
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse=False,handle_unknown='ignore'), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X1))
pk.dump(ct, file)
from sklearn.neighbors import KNeighborsClassifier
classifier1 = KNeighborsClassifier(n_neighbors = 35,weights='uniform', metric = 'minkowski', p = 2)
classifier1.fit(X, y)
pk.dump(classifier1, file)
file.close()

dataset_test['rank']=1

file = open("models.pkl", "rb")
trained_encoder = pk.load(file)  #Pickle file first load the OneHotEncoder 
trained_model_for_prediction = pk.load(file)
X_test_encoded=np.array(trained_encoder.transform(dataset_test))
y_predict=trained_model_for_prediction.predict(X_test_encoded)
dataset_test['DaystoPay']=y_predict
print(dataset_test)
file.close()
y_pred=classifier1.predict(X)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import average_precision_score,classification_report
from sklearn.metrics import f1_score
#average_precision = average_precision_score(y_test_in, y_pred)
cm = confusion_matrix(y,y_pred)
print( accuracy_score(y,y_pred))
print(classification_report(y,y_pred))

Payer                object
Debit_CreditInd.     object
DaystoPay           float64
dtype: object


C:\Users\deesaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


      Payer  Debit_CreditInd.  rank  DaystoPay
0    100030                 0     1        7.0
1    100030                 1     1       14.0
2  70000010                 1     1       20.0
3    300767                 1     1       20.0
4    100040                 0     1        8.0
5    100059                 0     1        7.0
6    301096                 1     1       20.0
7    301097                 1     1        9.0
8    301136                 1     1       10.0
9  70000000                 1     1       20.0
0.919646173064587


C:\Users\deesaw\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    -43902.0       0.00      0.00      0.00         1
    -43899.0       0.00      0.00      0.00         1
    -43898.0       0.00      0.00      0.00         1
    -43896.0       0.00      0.00      0.00         1
    -43893.0       0.00      0.00      0.00         1
    -43887.0       0.00      0.00      0.00         1
    -43886.0       0.00      0.00      0.00         1
    -43885.0       0.24      1.00      0.38         9
    -43881.0       0.00      0.00      0.00         1
    -43878.0       0.00      0.00      0.00         1
    -43874.0       0.00      0.00      0.00         1
    -43872.0       0.00      0.00      0.00         1
    -43871.0       0.00      0.00      0.00         5
    -43868.0       0.00      0.00      0.00         3
    -43864.0       0.00      0.00      0.00         6
    -43856.0       0.00      0.00      0.00         1
    -43851.0       0.00      0.00      0.00         1
    -43850.0       0.00    

In [ ]:
      Payer  Debit_CreditInd.  rank  DaystoPay
0    100030                 0     1        7.0
1    100030                 1     1       39.0
2  70000010                 1     1        0.0
3    300767                 1     1        0.0
4    100040                 0     1       22.0
5    100059                 0     1       11.0
6    301096                 1     1        4.0
7    301097                 1     1       16.0
8    301136                 1     1        2.0
9  70000000                 1     1        0.0